## Making Inferences in the ASIA graph

Notebook layout:
1. Load ASIA model
2. Make inferences

## Graph
![ASIA Graph](ASIA.jpg)

## 1. Load ASIA Model

In [1]:
# import essential libraries to manipulate data
import numpy as np
import pandas as pd

# import pgmpy functions
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load model
data = BIFReader('asia.bif')
asia_model = data.get_model()

In [3]:
# print nodes of the model
asia_model.nodes()

NodeView(('asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp'))

In [4]:
# print edges of the model
asia_model.edges()

OutEdgeView([('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')])

In [5]:
# get model parameters
asia_model.get_cpds()

[<TabularCPD representing P(asia:2) at 0x2529754be88>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0x2529754b9c8>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x2529754bbc8>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x2529754b788>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0x2529754b248>,
 <TabularCPD representing P(smoke:2) at 0x2529758e748>,
 <TabularCPD representing P(tub:2 | asia:2) at 0x25297532b88>,
 <TabularCPD representing P(xray:2 | either:2) at 0x25297532ac8>]

In [6]:
# print the CPDs by iterating through them
for cpd in asia_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of asia:
+-----------+------+
| asia(yes) | 0.01 |
+-----------+------+
| asia(no)  | 0.99 |
+-----------+------+
CPD of bronc:
+------------+------------+-----------+
| smoke      | smoke(yes) | smoke(no) |
+------------+------------+-----------+
| bronc(yes) | 0.6        | 0.3       |
+------------+------------+-----------+
| bronc(no)  | 0.4        | 0.7       |
+------------+------------+-----------+
CPD of dysp:
+-----------+-------------+------------+-------------+------------+
| bronc     | bronc(yes)  | bronc(yes) | bronc(no)   | bronc(no)  |
+-----------+-------------+------------+-------------+------------+
| either    | either(yes) | either(no) | either(yes) | either(no) |
+-----------+-------------+------------+-------------+------------+
| dysp(yes) | 0.9         | 0.8        | 0.7         | 0.1        |
+-----------+-------------+------------+-------------+------------+
| dysp(no)  | 0.1         | 0.2        | 0.3         | 0.9        |
+-----------+-------------+----

In [7]:
# get independencies
asia_model.get_independencies()

(asia _|_ smoke, lung, bronc)
(asia _|_ xray | either)
(asia _|_ smoke, lung | bronc)
(asia _|_ smoke, bronc | lung)
(asia _|_ lung, bronc | smoke)
(asia _|_ either, smoke, bronc, dysp, lung, xray | tub)
(asia _|_ dysp, xray | either, bronc)
(asia _|_ xray | either, dysp)
(asia _|_ smoke, xray, bronc, dysp | either, lung)
(asia _|_ xray, bronc, dysp | either, smoke)
(asia _|_ smoke, bronc, dysp, lung, xray | either, tub)
(asia _|_ smoke | lung, bronc)
(asia _|_ lung | smoke, bronc)
(asia _|_ either, smoke, dysp, lung, xray | tub, bronc)
(asia _|_ either, smoke, bronc, lung, xray | tub, dysp)
(asia _|_ bronc | smoke, lung)
(asia _|_ either, smoke, bronc, dysp, xray | tub, lung)
(asia _|_ smoke, bronc | lung, xray)
(asia _|_ either, bronc, dysp, lung, xray | smoke, tub)
(asia _|_ bronc | smoke, xray)
(asia _|_ either, smoke, bronc, dysp, lung | tub, xray)
(asia _|_ xray | either, bronc, dysp)
(asia _|_ smoke, dysp, xray | either, lung, bronc)
(asia _|_ dysp, xray | either, smoke, bronc)


## 2. Make Inferences

**Q1.** What is the probability of a person having bronchitis given that he/she doesn't smoke?

**Q2.** What is the probability that the person is a smoker given that their x-ray result is positive

**Q3.** Consider the following two probabilities:

1. The probability that a person doesn't have dyspnea given that he has bronchitis. 
2. The probability that a person doesn't have dyspnea given that he has bronchitis and doesn't have lung cancer.

By what percent does the probability increase or decrease from situation 1 to situation 2?

In [8]:
VariableElimination(asia_model).query(variables=['bronc'], evidence={'smoke': 0})['bronc']

KeyError: 0